In [ ]:
data_dir = ".\\data\\"

# SLC-2445, vac, 72hrs
filename = "2018-07-27-14-45-18.tdms"
keyence_filename= None

# SLC-1730, vac, 24hrs
#filename = "2018-07-26-14-39-24.tdms"
#keyence_filename= None

# SLC-1730, vac, 48hrs
#filename = "2018-07-30-16-19-44.tdms"
#keyence_filename= None

# SLC-2445, air, 30hrs, keyence
#filename = "2018-08-30-10-01-54.tdms"
#keyence_filename = "2018-08-30-10-03-14-KEYENCE.tdms"

# SLC-1730, air, 65hrs??, keyence
#filename = "2018-08-31-15-46-55.tdms"
#keyence_filename = "2018-08-31-15-46-59-KEYENCE.tdms"

#keyence_filename = None

#duty test SLC-1730
filename = "2018-09-27-09-10-21.tdms"
keyence_filename = None

#duty test SLC-2445
#filename = "2018-10-01-09-11-39.tdms"
#keyence_filename = None

In [ ]:
import batch_utils
import data_tools

In [ ]:
from ipywidgets import FloatProgress
from IPython.display import display

import numpy as np
from pprint import pprint
import datetime
from time import time

from importlib import reload
reload(batch_utils)
reload(data_tools)

data, info = batch_utils.load_data(filename, keyence_filename, suffix="seq") 
pprint(info)

batch_utils.print_lengths(data, "Dataset:")

batch_utils.print_structure(data)

In [ ]:
if keyence_filename is not None:
    k_sum = 0.
    k_count = 0

    for k in data['set_point']['avg_keyence']:
        if k is not None and not np.isnan(k):
            k_sum += k
            k_count += 1

    k_offs = k_sum / k_count

    print("Offsetting keyence values by", k_offs)

    def subtract(arr, val):
        out = list()
        for a in arr:
            try:
                new = a - val
            except TypeError:
                new = None
            out.append(new)
        return out

In [ ]:
if keyence_filename is not None:
    data['set_point']['avg_keyence'] = subtract(data['set_point']['avg_keyence'], k_offs)
    data['keyence']['data1'] = subtract(data['keyence']['data1'], k_offs)
    data['keyence']['set_point'] = subtract(data['keyence']['set_point'], k_offs)

In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *

init_notebook_mode(connected=True)

plotter = Scatter

def plot(l=None, plot=[0,-1], squash=True):
    if l is not None:
        limits = [data['set_point']['time'][l[0]], data['set_point']['time'][l[1]]]
        raw = dict(zip(data.keys(), [data_tools.truncate(limits, data[k], find_mode={'seek':False}) for k in data]))
        text = ["i={}".format(i+l[0]) for i in range(len(raw['set_point']['time']))]
    else:
        raw = data
        text = ["i={}".format(i) for i in range(len(raw['set_point']['time']))]
        
    # Drawing custom histograms is wayyy less data intensive!!
    def hist(points, bins=100, **kwargs):
        h, b = np.histogram(list(filter(lambda a: a is not None, points)), bins)
        return Bar(y=[(i+j)/2 for i, j in zip(b, b[1:])], 
                   x=h, orientation='h', 
                   text=["{} to {}".format(i,j) for i, j in zip(b, b[1:])],
                   #width=np.ones(len(h))*(b[1]-b[0]), 
                   **kwargs)
    
    def running_mean(x, N):
        cumsum = np.cumsum(np.insert(x, 0, 0)) 
        return (cumsum[N:] - cumsum[:-N]) / float(N)
    
    # Running averages for temperature
    raw['temperature']['avg_stage'] = running_mean(raw['temperature']['stage'], 1620)
    raw['temperature']['avg_base']  = running_mean(raw['temperature']['base'],  1620)
    raw['temperature']['avg_amb']   = running_mean(raw['temperature']['amb'],   1620)
    
    # split the keyence into up and down
    if keyence_filename is not None:
        raw['set_point']['avg_keyence(1)'] = raw['set_point']['avg_keyence'][::2]
        raw['set_point']['avg_keyence(2)'] = raw['set_point']['avg_keyence'][1::2]
        raw['set_point']['time(1)'] = raw['set_point']['time'][::2]
        raw['set_point']['time(2)'] = raw['set_point']['time'][1::2]
    
    # squash the data?
    # 10000 points per graph seems okay??
    
    if squash:
        squashed = dict()

        for k in raw:
            squash_interval = int(np.floor(len(raw[k]['time'])/2000))
            
            if squash_interval == 0:
                print("Can't squash", k)
                squashed[k] = raw[k]
            else:

                print("Squashing", k, "at", squash_interval)
                #print(sum([len(d[k][l] for l in d[k])]))

                r = list(range(0, len(raw[k]['time']), squash_interval))
                r.append(len(raw[k]['time'])-1)

                squashed[k] = dict()

                #print(r)

                if k == 'set_point':
                    text = list(text[::squash_interval])

                for l in raw[k]:
                    print(l)

                    if l.startswith('time'):
                        squashed[k][l] = list(raw[k][l][::squash_interval])
                    else:
                        squashed[k][l] = list()
                        for i, j in zip(r, r[1:]):
                            if all([z is None for z in raw[k][l][i:j]]):
                                squashed[k][l].append(None)
                            else:
                                try:
                                    squashed[k][l].append(np.mean(raw[k][l][i:j]))
                                except TypeError:
                                    print("Cant process [{}:{}]".format(i,j))
                                    #print(raw[k][l][i:j])
                                    squashed[k][l].append(None)

                print("Squashed", k)
                #print(sum([len(d[k][l] for l in d[k])]))

        d = squashed
    else:
        d = data
                
    #if keyence_filename is not None:
    #    foo, bins = np.histogram(raw['set_point']['avg_keyence'] + raw['set_point']['avg_position'], bins=1000)
    #else:
    #    foo, bins = np.histogram(raw['set_point']['avg_position'], bins=1000)

    plots = [
        plotter(x=d['set_point']['time'], y=d['set_point']['data'], name="Set Point", line={'shape':'hv'}, text=text),
        
        plotter(x=d['position']['time'],  y=d['position']['data'], name="Stage Position"),
        plotter(x=d['position']['time'],  y=np.ma.MaskedArray(d['position']['data'], mask=d['position']['mask']), 
                name="Stage Position Masked"),
        
        plotter(x=d['position']['time'], y=d['position']['set_point'], name="Stage Error"),
        plotter(x=d['position']['time'], y=np.ma.MaskedArray(d['position']['set_point'], mask=d['position']['mask']), 
                name="Stage Error Masked"),      

        plotter(x=d['set_point']['time'], y=d['set_point']['avg_position'], name="Stage Error Average", line={'shape':'hv'}, text=text),

        hist(raw['set_point']['avg_position'], name="Stage Error Average", xaxis='x2'),
        
        plotter(x=d['temperature']['time'], y=d['temperature']['stage'], name="Stage Temperature",   yaxis='y2'),
        plotter(x=d['temperature']['time'], y=d['temperature']['base'],  name="Base Temperature",    yaxis='y2'),
        plotter(x=d['temperature']['time'], y=d['temperature']['amb'],   name="Ambient Temperature", yaxis='y2'),
        
        plotter(x=d['temperature']['time'], y=d['temperature']['avg_stage'], name="Stage Temp Avg",   yaxis='y2'),
        plotter(x=d['temperature']['time'], y=d['temperature']['avg_base'],  name="Base Temp Avg",    yaxis='y2'),
        plotter(x=d['temperature']['time'], y=d['temperature']['avg_amb'],   name="Ambient Temp Avg", yaxis='y2'),
        
        plotter(x=0, y=0, name='Dummy'),
    ]
    
    if keyence_filename is not None:
        plots = plots + [
        plotter(x=d['keyence']['time'],  y=d['keyence']['data1'], name="Measured Position"),
        
        plotter(x=d['keyence']['time'],  y=np.ma.MaskedArray(d['keyence']['data1'], mask=d['keyence']['mask']), 
               name="Measured Position Masked"),
        plotter(x=d['keyence']['time'], y=d['keyence']['set_point'], name="Measured Error"),
        plotter(x=d['keyence']['time'], y=np.ma.MaskedArray(d['keyence']['set_point'], mask=d['keyence']['mask']), 
                name="Measured Error Masked"),
        
        plotter(x=d['set_point']['time'], y=d['set_point']['avg_keyence'], name="Measured Error Average", line={'shape':'hv'}, text=text),
        plotter(x=d['set_point']['time(1)'], y=d['set_point']['avg_keyence(1)'], name="Measured Error Average(1)", line={'shape':'hv'}, text=text),
        plotter(x=d['set_point']['time(2)'], y=d['set_point']['avg_keyence(2)'], name="Measured Error Average(2)", line={'shape':'hv'}, text=text),
        

        plotter(x=0, y=0, name='Dummy'),
    ]
        
    try:
        plots.append(hist(raw['set_point']['avg_keyence'], name="Measured Error Average", xaxis='x2'))
        plots.append(hist(raw['set_point']['avg_keyence'][::2], name="Measured Error Average (1)", xaxis='x2'))
        plots.append(hist(raw['set_point']['avg_keyence'][1::2], name="Measured Error Average (2)", xaxis='x2'))
    except ValueError:
        print("Can't do the histograms!")

    # Too much data, don't plot the un-aggregated stuff!
    #if sum([len(d[e]['time']) for e in d]) > 7000:
    #    print("Too much data!")
    #    plots = plots[5:9]
    
    print(sum([len(d[e]['time']) for e in d]))
    
    if plots[1] == -1:
            plots = plots[plot[0]:]
    else:
        plots = plots[plot[0]:plot[1]]

    layout = Layout(
        width = 800,
        height = 800,
        title = filename,
        xaxis = dict(
            title = "Time",
            domain=[0.0, 0.7],
        ),
        yaxis = dict(
            title = "Position (mm)",
            position=0.0,
        ),
        xaxis2 = dict(
            title = "Log Prob",
            type = "log",
            domain=[0.8, 1],
        ),
        yaxis2 = dict(
            title = "Temperature (°C)",
            overlaying='y',
            side='right',
        ),
        legend = dict(
            orientation="h",
        ),
        bargap=0,
        barmode='overlay',
    )

    fig = Figure(data=plots, layout=layout)
    iplot(fig, filename=filename)

In [ ]:
print(len(data['set_point']['time']))

if filename == "2018-08-30-10-01-54.tdms":
    plot([0, 50000], [5, -1])

if filename == "2018-08-31-15-46-55.tdms":
    plot([0,73900], [5, -1])
    
if filename == "2018-07-27-14-45-18.tdms":
    plot([0, 100000], [5, -1])
    
if filename == "2018-09-27-09-10-21.tdms":
    plot([0, 61000], [5, -1])
    plot([61000, -1], [5, -1])
    
if filename == "2018-10-01-09-11-39.tdms":
    plot([0, 68000], [5, -1])
    plot([68000, -1], [5, -1])

In [ ]:

if filename == "2018-07-27-14-45-18.tdms":
    plot([37100, 37350]) #72hr
    
if filename == "2018-07-30-16-19-44.tdms":
    plot([69000, 69100]) #48hr

if filename == "2018-08-30-10-01-54.tdms":
    plot([15800, 15900])
    
if filename == "2018-08-31-15-46-55.tdms":
    plot([100, 150])
    
if filename == "2018-09-27-09-10-21.tdms":
    plot([0, 2500])

In [ ]:
if filename == "2018-07-27-14-45-18.tdms":
    plot([48550, 48700]) #72hr
    
if filename == "2018-07-30-16-19-44.tdms":
    plot([75600, 75700]) #48hr
    
if filename == "2018-08-30-10-01-54.tdms":
    plot([56300, 56400])
    
if filename == "2018-08-31-15-46-55.tdms":
    plot([41950, 42000]) 
    
    
if filename == "2018-09-27-09-10-21.tdms":
    plot([300, 400])
    plot([-300, -100])

In [ ]:
def plot_time(k=None, l=None, mode='plotly'):
    if l is not None:
        limits = [data['set_point']['time'][l[0]], data['set_point']['time'][l[1]]]
        d = dict(zip(data.keys(), [data_tools.truncate(limits, data[k], find_mode={'seek':False}) for k in data]))
        print("Using limits - don't trust i!")
    else:
        d = data
    
    if mode=='plotly':
        if k is None:
            plots = [
                plotter(x=data[key]['time'], y=[(b-a).total_seconds() for a, b in zip(data[key]['time'], data[key]['time'][1:])], 
                        name=key, text=["i={}".format(i) for i in range(len(data[key]['time']))]) for key in data
            ]
        else:
            plots = [
                plotter(x=data[k]['time'], y=[(b-a).total_seconds() for a, b in zip(data[k]['time'], data[k]['time'][1:])], 
                        name=k, text=["i={}".format(i) for i in range(len(data[k]['time']))]),
            ]

        layout = Layout(
            width = 800,
            height = 800,
            title = filename,
            xaxis = dict(
                title = "Time",
            ),
            yaxis = dict(
                title = "Δ Time (s)",
            ),
            legend = dict(
                orientation="h",
            )
        )

        fig = Figure(data=plots, layout=layout)
        iplot(fig, filename=filename)
        
    elif mode == 'stats':      
        time_diff = [(b-a) for a, b in zip(data[k]['time'], data[k]['time'][1:])]
        
        pprint([
            ("Max ", np.max(time_diff).total_seconds(), data[k]['time'][time_diff.index(np.max(time_diff))]),
            ("Mean", np.mean(time_diff).total_seconds()),
            ("Min ", np.min(time_diff).total_seconds(), data[k]['time'][time_diff.index(np.min(time_diff))])
        ])

In [ ]:
plot_time(k="set_point", mode="stats")

In [ ]:
if keyence_filename == "2018-08-30-10-03-14-KEYENCE.tdms":
    while True:

        a = data['set_point']['avg_keyence']
        b = -0.007522212 

        c = []

        for i in range(len(a)):
            try:
                d = a[i]-b
            except TypeError:
                d = 999

            c.append(d)

        j = (c.index(min(c)))

        if j == 0:
            print("Done")
            break

        print(j, min(c), data['set_point']['avg_keyence'][j])
        data['set_point']['avg_keyence'][j] = None

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as md
import matplotlib.ticker as tick

def box_plot(l=None, plot=[0,-1], times=None, boxes=True):
    
    if l is not None:
        limits = [data['set_point']['time'][l[0]], data['set_point']['time'][l[1]]]
        raw = dict(zip(data.keys(), [data_tools.truncate(limits, data[k], find_mode={'seek':False}) for k in data]))
        text = ["i={}".format(i+l[0]) for i in range(len(raw['set_point']['time']))]
    else:
        raw = data
        text = ["i={}".format(i) for i in range(len(raw['set_point']['time']))]
        
    # split the keyence into up and down
    if keyence_filename is not None:
        # sanitise keyence values - there are some nan/None in there!!
        
        san = {'time':list(), 'avg':list()}
        
        for t, a in zip(raw['set_point']['time'], raw['set_point']['avg_keyence']):
            if a is not None and np.isfinite(a):
                san['time'].append(t)
                san['avg'].append(a)

        
        #raw['set_point']['avg_keyence(1)'] = san['avg'][::2]
        #raw['set_point']['avg_keyence(2)'] = san['avg'][1::2]
        #raw['set_point']['time(1)'] = san['time'][::2]
        #raw['set_point']['time(2)'] = san['time'][1::2]
        
        raw['set_point']['avg_keyence(1)'] = data['set_point']['avg_keyence'][::2]
        raw['set_point']['avg_keyence(2)'] = data['set_point']['avg_keyence'][1::2]
        raw['set_point']['time(1)'] = data['set_point']['time'][::2]
        raw['set_point']['time(2)'] = data['set_point']['time'][1::2]
        
        def sanitize(times, values):
            new_times = []
            new_values = []
            for t, a in zip(times, values):
                if a is not None and np.isfinite(a):
                    new_times.append(t)
                    new_values.append(a)
                    
            return new_times, new_values
        
        raw['set_point']['time(1)'], raw['set_point']['avg_keyence(1)'] = sanitize(data['set_point']['time'][::2], 
                                                                                   data['set_point']['avg_keyence'][::2])
        raw['set_point']['time(2)'], raw['set_point']['avg_keyence(2)'] = sanitize(data['set_point']['time'][1::2], 
                                                                                   data['set_point']['avg_keyence'][1::2])
        
        raw['position']['time'], raw['position']['data'] = sanitize(data['position']['time'], 
                                                                                   data['position']['data'])
                
        #print(any([np.isnan(r) for r in raw['set_point']['avg_keyence(1)']]))
        #print(any([np.isnan(r) for r in raw['set_point']['avg_keyence(2)']]))
    
    d = raw
    
    # List of each value's group - these are datetimes!!
    groups = []
    
    # Data grouped into bunches for each time
    grouped_data_pos = []
    grouped_data_key1 = []
    grouped_data_key2 = []
    
    
    if times is None:
        # not fully implemented...
        times = []
        
        interval = int(len(d['set_point']['time'])/12)

        for i in range(int(len(d['set_point']['time'])/interval)):
            groups.extend([d['set_point']['time'][i*interval],]*interval)
            times.append(d['set_point']['time'][i*interval])
            
    else:
        for j, (t, u) in enumerate(zip(times, times[1:])):
            grouped_data_pos.append(list())
            #grouped_data_key.append(list())
            grouped_data_key1.append(list())
            grouped_data_key2.append(list())
            
            for i, s in enumerate(d['set_point']['time']):
                if s >= t and s < u:
                    groups.append(t)
                    grouped_data_pos[-1].append(d['set_point']['avg_position'][i])
                    #grouped_data_key[-1].append(d['set_point']['avg_keyence'][i])
            
            for i, s in enumerate(d['set_point']['time(1)']):
                if s >= t and s < u:
                    groups.append(t)
                    grouped_data_key1[-1].append(d['set_point']['avg_keyence(1)'][i])
                    
            for i, s in enumerate(d['set_point']['time(2)']):
                if s >= t and s < u:
                    groups.append(t)
                    grouped_data_key2[-1].append(d['set_point']['avg_keyence(2)'][i])
    
    ### Start plotting!!
        
    half = (times[1].timestamp() - times[0].timestamp())/2
    
    labels = [t.strftime("%H:%M") + " to " + u.strftime("%H:%M") for (t, u) in zip(times, times[1:])]
    time_labels = [t.strftime("%H:%M") for t in d['temperature']['time']]
    positions = [t.timestamp()+half for t in times[:-1]]
    
    flierprops = dict(marker='x', markeredgecolor='red')
    meanprops = dict(marker='o', markeredgecolor='black', markerfacecolor='None')
    medianprops = dict(linestyle='-', linewidth=2.5, color='black')
    
    whis = 'range'
    widths = 3600/2
    
    #fig, (ax1, ax2, ax3) = plt.subplots(3,1, gridspec_kw = {'height_ratios':[1, 1, 1]}, dpi=100, figsize=[8,8])
    fig = plt.figure(dpi=100, figsize=[8,8])
    
    ax1 = plt.subplot(311)   
    ax2 = plt.subplot(312, sharex=ax1)
    ax3 = plt.subplot(313, sharex=ax1)
    
    ax1.plot([t.timestamp() for t in d['set_point']['time']], d['set_point']['avg_position'], color='#2ca02c', alpha=0.4, label="Feedback")
    
    ax1.boxplot(grouped_data_pos, labels=labels, positions=positions, widths=widths,
                flierprops=flierprops, showmeans=True, whis=whis, meanprops=meanprops, medianprops=medianprops)
    
    ax2.plot([t.timestamp() for t in d['set_point']['time(1)']], d['set_point']['avg_keyence(1)'], alpha=0.4, label="Measured (1)")
    ax2.plot([t.timestamp() for t in d['set_point']['time(2)']], d['set_point']['avg_keyence(2)'], alpha=0.4, label="Measured (2)")
    
    ax2.boxplot(grouped_data_key1, positions=positions,  widths=widths,
                flierprops=flierprops, showmeans=False, whis=whis, meanprops=meanprops, medianprops=medianprops)
    ax2.boxplot(grouped_data_key2, positions=positions,  widths=widths,
                flierprops=flierprops, showmeans=False, whis=whis, meanprops=meanprops, medianprops=medianprops)
    
    ax3.plot([t.timestamp() for t in d['temperature']['time']], d['temperature']['stage'], alpha=0.4, label="Stage")
    
    ax1.grid()
    plt.setp(ax1.get_xticklabels(), visible=False)
    ax2.grid()
    plt.setp(ax2.get_xticklabels(), visible=False)
    ax3.grid()
    
    #ax3.set_xlabels([datetime.fromtimestamp(t).strftime("%H:%M") for t in ax3.get_xlabels()])
    
    for t in ax3.get_xticklabels():
        t.set_rotation(90)
        
    def fmt(t, pos):
        return datetime.datetime.fromtimestamp(t).strftime("%H:%M")
    
    formatter = tick.FuncFormatter(fmt)
    ax3.xaxis.set_major_formatter(formatter)
    
    ax1.set_xlim(times[0].timestamp(), times[-1].timestamp())
    
    fig.suptitle("Positioning accuracy over time \n({})".format(filename))
    ax1.legend()
    ax2.legend()
    ax3.legend()

    ax1.ticklabel_format(style='sci', axis='y', scilimits=(0,0), useMathText=True)
    ax2.ticklabel_format(style='sci', axis='y', scilimits=(0,0), useMathText=True)
    
    ax1.set_ylabel("Deviation (mm)")
    ax2.set_ylabel("Deviation (mm)")
    ax3.set_ylabel("Temperature (°C)")
    
    plt.show()

In [ ]:
base = datetime.datetime.combine(data['set_point']['time'][0].date(), datetime.time(data['set_point']['time'][0].time().hour-1, 30), 
                                 tzinfo=data['set_point']['time'][0].tzinfo)

duration = data['set_point']['time'][-1] - data['set_point']['time'][0]
                                                                       
duration = np.ceil(duration.total_seconds()/3600)

times = [base + datetime.timedelta(hours=n) for n in np.linspace(0, duration, duration+1)]

box_plot(times=times)

In [ ]:
def plot_violin(l=None, plot=[0,-1], times=None, boxes=True):
    if l is not None:
        limits = [data['set_point']['time'][l[0]], data['set_point']['time'][l[1]]]
        raw = dict(zip(data.keys(), [data_tools.truncate(limits, data[k], find_mode={'seek':False}) for k in data]))
        text = ["i={}".format(i+l[0]) for i in range(len(raw['set_point']['time']))]
    else:
        raw = data
        text = ["i={}".format(i) for i in range(len(raw['set_point']['time']))]
    
    d = raw
    
    if times is None:
        interval = int(len(d['set_point']['time'])/12)

        groups = []
        for i in range(int(len(d['set_point']['time'])/interval)):
            groups.extend([d['set_point']['time'][i*interval],]*interval)
    else:
        groups = []
        
        j = 0
        
        for t, u in zip(times, times[1:]):
            for s in  d['set_point']['time']:
                if s >= t and s < u:
                    groups.append(t)

    fig = {
        "data": [
            {
            "type": 'violin',
            "x": groups,
            "y": d['set_point']['avg_position'],
            "legendgroup": 'S',
            "scalegroup": 'S',
            "yaxis": "y1",
            "side": 'negative',
            "name": "Stage Error",
            "box": {
                "visible": boxes
            },
            "line": {
            #    "color": 'black'
            },
            "meanline": {
                "visible": True
            },
            #"fillcolor": '#8dd3c7',
            #"opacity": 0.6,
        },
        {
            "type": 'violin',
            "x": groups,
            "y": d['set_point']['avg_keyence'],
            "legendgroup": 'M',
            "scalegroup": 'M',
            "yaxis": "y2",
            "side": 'positive',
            "name": "Measured Error",
            "box": {
                "visible": boxes
            },
            "line": {
            #    "color": 'black'
            },
            "meanline": {
                "visible": True
            },
            #"fillcolor": '#8dd3c7',
            #"opacity": 0.6,
        },
        ],
        "layout" : {
            "title": "",
            "xaxis": {
                "domain": [0.1, 1],
            },
            "yaxis": {
                "title": "Stage error (mm)",
                "position": 0,
            },
            "yaxis2": {
                "overlaying": "y",
                "title": "Measured error (mm)",
                "position": 0.1,
            },
            "violingap": 0,
            "violingroupgap": 0,
            "violinmode": "overlay"
        }
    }
    
    iplot(fig, filename=filename, validate=False)

In [ ]:
base = datetime.datetime.combine(data['set_point']['time'][0].date(), datetime.time(15), 
                                 tzinfo=data['set_point']['time'][0].tzinfo)

times = [base + datetime.timedelta(hours=n) for n in np.linspace(0, 66, 11+1)]

plot_violin(times=times)

In [ ]:
base = datetime.datetime.combine(datetime.date(2018, 9, 2), datetime.time(20), 
                                 tzinfo=data['set_point']['time'][0].tzinfo)

times = [base + datetime.timedelta(hours=n) for n in np.linspace(0, 6, 25)]

plot_violin(times=times)

In [ ]:
base = datetime.datetime.combine(datetime.date(2018, 9, 2), datetime.time(23), 
                                 tzinfo=data['set_point']['time'][0].tzinfo)

times = [base + datetime.timedelta(hours=n) for n in np.linspace(0, 4, 6*4+1)]

plot_violin(times=times)